In [1]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip install python-dotenv
!pip -qqq install flashrank==0.2.4 --progress-bar off


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-aiplatform 1.59.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-bigquery-storage 2.25.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but

In [3]:
import os
import textwrap
from pathlib import Path
from google.colab import userdata
import pandas as pd
import numpy as np
import qdrant_client
import requests
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse
from dotenv import load_dotenv

In [ ]:
import os
os.environ['GROQ_API_KEY'] = input("Enter your GROQ API Key: ")
def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [5]:

import getpass  # Import getpass for secure input
instruction = """The provided documents provide detailed financial information about the company's performance for a specific quarter.
It includes unaudited financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions



 """

# Get API key using getpass for secure input
api_key = getpass.getpass("Enter your API Key: ")




parser = LlamaParse(
    api_key=api_key,  # Use the entered API key
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

pdf_files = [f for f in os.listdir("/content/data") if f.endswith(".pdf")]
llama_parse_documents = []

for pdf_file in pdf_files:
    file_path = os.path.join("/content/data", pdf_file)
    document = await parser.aload_data(file_path)
    llama_parse_documents.append(document)



Enter your API Key: ··········
Started parsing the file under job_id 1d9b0f1a-ddce-41b6-bb4e-839e040522ba


In [6]:
parsed_doc = llama_parse_documents[0][0] # Access the first Document object in the list
Markdown(parsed_doc.text[:4096])

# Meta Reports Fourth Quarter and Full Year 2023 Results; Initiates Quarterly Dividend

MENLO PARK, Calif. – February 1, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter and full year ended December 31, 2023.

"We had a good quarter as our community and business continue to grow," said Mark Zuckerberg, Meta founder and CEO. "We've made a lot of progress on our vision for advancing AI and the metaverse."

# Fourth Quarter and Full Year 2023 Financial Highlights

|In millions, except percentages and per share amounts|Three Months Ended December 31, 2023|Three Months Ended December 31, 2022|% Change|Twelve Months Ended December 31, 2023|Twelve Months Ended December 31, 2022|% Change|
|---|---|---|---|---|---|---|
|Revenue|$40,111|$32,165|25%|$134,902|$116,609|16%|
|Costs and expenses|$23,727|$25,766|(8)%|$88,151|$87,665|1%|
|Income from operations|$16,384|$6,399|156%|$46,751|$28,944|62%|
|Operating margin|41%|20%| |35%|25%| |
|Provision for income taxes|$2,791|$1,497|86%|$8,330|$5,619|48%|
|Effective tax rate|17%|24%| |18%|19%| |
|Net income|$14,017|$4,652|201%|$39,098|$23,200|69%|
|Diluted earnings per share (EPS)|$5.33|$1.76|203%|$14.87|$8.59|73%|

# Fourth Quarter and Full Year 2023 Operational and Other Financial Highlights

- Family daily active people (DAP) – DAP was 3.19 billion on average for December 2023, an increase of 8% year-over-year.
- Family monthly active people (MAP) – MAP was 3.98 billion as of December 31, 2023, an increase of 6% year-over-year.
- Facebook daily active users (DAUs) – DAUs were 2.11 billion on average for December 2023, an increase of 6% year-over-year.
- Facebook monthly active users (MAUs) – MAUs were 3.07 billion as of December 31, 2023, an increase of 3% year-over-year.
- Ad impressions and price per ad – In the fourth quarter of 2023, ad impressions delivered across our Family of Apps increased by 21% year-over-year and the average price per ad increased by 2% year-over-year. For the full year 2023, ad impressions increased by 28% year-over-year and the average price per ad decreased by 9% year-over-year.
- Revenue – Revenue was $40.11 billion and $134.90 billion, an increase of 25% and 16% year-over-year for the fourth quarter and full year 2023, respectively. Had foreign exchange rates remained constant with the same periods of 2022, revenue would have been $816 million and $374 million lower, an increase of 22% and 15% on a constant currency basis for the fourth quarter and full year 2023, respectively.
- Costs and expenses – Total costs and expenses were $23.73 billion and $88.15 billion for the fourth quarter and full year 2023, a decrease of 8% and an increase of 1% year-over-year, respectively. Restructuring charges included in costs and expenses were $1.15 billion and $3.45 billion for the fourth quarter and full year 2023, respectively.
- Capital expenditures – Capital expenditures, including principal payments on finance leases, were $7.90 billion and $28.10 billion for the fourth quarter and full year 2023, respectively.
- Share repurchases – We repurchased $6.32 billion and $20.03 billion of our Class A common stock in the fourth quarter and full year 2023, respectively. As of December 31, 2023, we had $30.93 billion available and authorized for repurchases. We also announced a $50 billion increase in our share repurchase authorization today.
- Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were $65.40 billion as of December 31, 2023. Free cash flow was $11.50 billion and $43.01 billion for the fourth quarter and full year 2023, respectively.
- Long-term debt – Long-term debt was $18.39 billion as of December 31, 2023.
- Headcount – Headcount was 67,317 as of December 31, 2023, a decrease of 22% year-over-year.
---
# Meta Initiates Quarterly Dividend

Today, Meta's board of directors declared a cash dividend of $0.50 per share of our outstanding common stock (including both Class A common stock and Class B common stock), payable on March 26, 2024 to stockholders of record as 

In [7]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)


In [8]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

15

In [10]:
print(docs[11].page_content)

Three Months Ended December 31, 2023 Three Months Ended December 31, 2022 Twelve Months Ended December 31, 2023 Twelve Months Ended December 31, 2022 Cash flows from operating activities Net income $14,017 $4,652 $39,098 $23,200 Adjustments to reconcile net income to net cash provided by operating activities: Depreciation and amortization 3,172 2,376 11,178 8,686 Share-based compensation 3,424 3,008 14,027 11,992 Deferred income taxes -1,161 -1,173 131 -3,286 Impairment charges for facilities consolidation, net 1,091 1,805 2,432 2,218 Data center assets abandonment 7 1,341 -224 1,341 Other 124 572 635 641 Changes in assets and liabilities: Accounts receivable -2,843 -1,698 -2,399 231 Prepaid expenses and other current assets 700 854 559 162 Other assets -111 54 -80 -106 Accounts payable 595 876 51 210 Partners payable 76 102 -271 90 Accrued expenses and other current liabilities -350 1,303 5,352 4,210 Other liabilities 663 439 624 886 Net cash provided by operating activities 19,404 14

In [11]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [12]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [13]:
%%time
query = "What has always been the important innovation from Meta?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 312 ms, sys: 8.73 ms, total: 321 ms
Wall time: 324 ms


In [14]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

score: 0.6910220970732008
--------------------------------------------------------------------------------

text: Meta Reports Fourth Quarter and Full Year 2023 Results; Initiates Quarterly Dividend

MENLO PARK, Calif. – February 1, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter and full year ended December 31, 2023.

"We h

score: 0.5635992618422168
--------------------------------------------------------------------------------

text: Absent any changes to U.S. tax law, we expect our full-year 2024 tax rate to be in the mid-teens.

In addition, we continue to monitor the active regulatory landscape, including the increasing legal and regulatory headwinds in the EU and the U.S. tha

In [15]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 319 ms, sys: 1.82 ms, total: 321 ms
Wall time: 326 ms


In [16]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()


id: 8e4eb5415fab495ba9762a95e3daeb60

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

--------------------------------------------------------------------------------

id: 0c1448a240d848a890e944481fc2ba47

text: Meta Reports Fourth Quarter and Full Year 2023 Results; Initiates Quarterly Dividend

MENLO PARK, Calif. – February 1, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter and full year ended December 31, 2023.

"We h

--------------------------------------------------------------------------------

id: eeb989053cc24a10958024a12732a059

text: Absent any changes to U.S. tax law, we expect our full-year 2024 tax rate to be in the mid-teens.

In addition, we continue to monitor the active regulatory landscape, including the increa

In [17]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 72.9MiB/s]


In [18]:

%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)


Running pairwise ranking..
CPU times: user 3.49 s, sys: 285 ms, total: 3.77 s
Wall time: 3.77 s


3

In [19]:

for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()


id: 8e4eb5415fab495ba9762a95e3daeb60

text: About Meta

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further empowered billions around the worl

score: 0.8372957110404968
--------------------------------------------------------------------------------

id: 0c1448a240d848a890e944481fc2ba47

text: Meta Reports Fourth Quarter and Full Year 2023 Results; Initiates Quarterly Dividend

MENLO PARK, Calif. – February 1, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter and full year ended December 31, 2023.

"We h

score: 0.029711542651057243
--------------------------------------------------------------------------------

id: bda25eb78b29456c8f9e364066d37cae

text: Long-term debt – Long-term debt was $18.39 billion as of December 31, 2023.

Headcount – Headcount was 67,317 as of December 31, 2023,

In [20]:

llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [21]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)

In [ ]:
response = qa.invoke("Total assets and liabilities of Meta in this financial year")
print_response(response)

Running pairwise ranking..
**Total Assets and Liabilities of Meta**

As of December 31, 2023, the total assets of Meta are **$229,623 million**, and the total
liabilities are **$76,455 million**.

Here is a breakdown of the assets and liabilities:

**Assets:**

* Current assets: $85,365 million
* Non-current assets: $144,258 million
        + Non-marketable equity securities: $6,141 million
        + Property and equipment, net: $96,587 million
        + Operating lease right-of-use assets: $13,294 million
        + Intangible assets, net: $788 million
        + Goodwill: $20,654 million
        + Other assets: $6,794 million

**Liabilities:**

* Current liabilities: $31,960 million
* Non-current liabilities: $44,495 million
        + Operating lease liabilities, non-current: $17,226 million
        + Long-term debt: $18,385 million
        + Other liabilities: $8,884 million


In [ ]:
response = qa.invoke(" from revenue , Costs and expenses in 2023 at the end of 12 months calculate the Income from operations and the operating margin")
print_response(response)


Running pairwise ranking..
Based on the provided information, we can calculate the Income from operations and the operating
margin for 2023 at the end of 12 months as follows:

**Income from Operations:**

Revenue: $134,902 million (from the "Revenue" section)
Costs and Expenses: $88,155 million (from the "Costs and expenses" section)

Income from Operations = Revenue - Costs and Expenses
= $134,902 million - $88,155 million
= $46,747 million

**Operating Margin:**

Operating Margin = Income from Operations / Revenue
= $46,747 million / $134,902 million
= 34.7%

Additional helpful information:

* The Family of Apps segment contributed $62,871 million to the total income from operations, while
the Reality Labs segment had a loss of $16,120 million.
* The company's free cash flow for the full year 2023 was $43,010 million, indicating a strong cash
generation capability.
* The company has a significant amount of cash, cash equivalents, and marketable securities,
totaling $65,400 million a

In [24]:
%%time
response = qa.invoke("What has always been the most important innovation by meta?")
print_response(response)

Running pairwise ranking..
Based on the provided information, I don't know what has always been the most important innovation
by Meta. The text does not explicitly state what the most important innovation is. However, it does
mention that "when Facebook launched in 2004, it changed the way people connect" and that Meta is
"moving beyond 2D screens toward immersive experiences like augmented and virtual reality to help
build the next evolution in social technology." This suggests that Facebook's launch and the
development of immersive experiences like AR and VR are significant innovations, but it does not
identify a single most important innovation.
CPU times: user 4.04 s, sys: 238 ms, total: 4.27 s
Wall time: 5.18 s
